# Ejemplo de ETL: Limpieza de Nulos y Outliers

Aquí realizaremos un flujo ETL típico:
1. Carga de datos del Padrón Electoral.
2. Detección y limpieza de valores nulos.
3. Detección y filtrado de outliers (ejemplo: edades fuera de rango lógico).
4. Transformación de columnas y guardado.


In [0]:
from pyspark.sql.functions import col, when

from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType

padron_schema  = StructType(fields=[
    StructField("CEDULA",IntegerType(),True), 
    StructField("CODELEC",IntegerType(),True),
    StructField("RELLENO",StringType(),True),
    StructField("FECHACADUC",DateType(),True),
    StructField("JUNTA",IntegerType(),True),
    StructField("NOMBRE",StringType(),True),
    StructField("1_APELLIDO",StringType(),True),
    StructField("2_APELLIDO",StringType(),True),
])

df = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(padron_schema)\
    .csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/PADRON_COMPLETO.csv")



In [0]:
# 1. Inspección inicial
df.printSchema()



In [0]:
df.select([col for col in df.columns]).show(5)



In [0]:
# 2. Detección de nulos
for c in df.columns:
    print(c, "->", df.filter(col(c).isNull()).count(), "nulos")



In [0]:
# 3. Limpieza: Eliminar filas con cédula o nombre nulo
df_clean = df.dropna(subset=["CEDULA", "NOMBRE"])



In [0]:
# 4. Supongamos que tenemos una columna EDAD y filtramos outliers
# (Si no hay, simular)
from pyspark.sql.functions import rand
df_clean = df_clean.withColumn("EDAD", (rand()*90+10).cast("int"))  # Simular edad



In [0]:
# Filtrar edades <18 o >100
df_clean = df_clean.filter((col("EDAD") >= 18) & (col("EDAD") <= 100))



In [0]:
# 5. Transformación: agregar columna "mayor_de_edad"
df_clean = df_clean.withColumn("mayor_de_edad", when(col("EDAD") >= 18, True).otherwise(False))

df_clean.select("CEDULA", "NOMBRE", "EDAD", "mayor_de_edad").show(5)

# Guardar resultado
df_clean.write.mode("overwrite").csv("/Volumes/big_data_ii_2025/spark_examples/spark_data//padron_limpio.csv", header=True)